# Feature Matching - Part 1

* Template Matching that we did before finds the part of the bigger image which exactly matches the smaller image. So smaller image is fully contained in the bigger image. We know it's there. We have its exact copy. E.g. we had exact copy image of the dog's face.


* Feature Matching:
   * extracts defining key features (corners, edges, contours) from an input/target image (an image we'll try to find in the bigger image)
   * using a distance calculation finds all the matches in another image; what's being matched are features but not 1:1 pixels like in Template Matching
   * no exact copy of the target image is required; input image can be some other dog's face image where, taken from a different angle etc...
   

* Feature Matching methods we'll be using here are:
   * Brute-Force Matching with ORB Descriptors
   * Brute-Force Matching with SIFT Descriptors and Ratio Test
   * FLANN-based Matcher
   
   
[Feature Matching](https://docs.opencv.org/master/dc/dc3/tutorial_py_matcher.html)
   
[Feature Detection and Description](https://docs.opencv.org/master/db/d27/tutorial_py_table_of_contents_feature2d.html)

**Keypoints** are the same thing as interest points. They are spatial locations, or points in the image that define what is interesting or what stand out in the image.

From [Meaning of Keypoints and Descriptors](https://answers.opencv.org/question/37985/meaning-of-keypoints-and-descriptors/):

**Descriptors**: they are the way to compare the keypoints. They summarize, in vector format (of constant length) some characteristics about the keypoints. For example, it could be their intensity in the direction of their most pronounced orientation. It's assigning a numerical description to the area of the image the keypoint refers to.

Some important things for descriptors are:
   * they should be independent of keypoint position. If the same keypoint is extracted at different positions (e.g. because of translation) the descriptor should be the same.
   * they should be robust against image transformations. Some examples are changes of contrast (e.g. image of the same place during a sunny and cloudy day) and changes of perspective (image of a building from center-right and center-left, we would still like to recognize it as a same building). Of course, no descriptor is completely robust against all transformations (nor against any single one if it is strong, e.g. big change in perspective). Different descriptors are designed to be robust against different transformations which is sometimes opposed to the speed it takes to calculate them.
   * they should be scale independent. The descriptors should take scale in to account. If the "prominent" part of the one keypoint is a vertical line of 10px (inside a circular area with radius of 8px), and the prominent part of another a vertical line of 5px (inside a circular area with radius of 4px) -- these keypoints should be assigned similar descriptors.

Now, that you calculated descriptors for all the keypoinst, you have a way to compare those keypoints. For a simple example of image matching (when you know the images are of the same object, and would like to identify the parts in different images that depict the same part of the scene, or would like to identify the perspective change between two images), you would compare every keypoint descriptor of one image to every keypoint descriptor of the other image. As the descriptors are vectors of numbers, you can compare them with something as simple as Euclidian distance. There are some more complex distances that can be used as a similarity measure, of course. But, in the end, you would say that the keypoints whose descriptors have the smallest distance between them are matches, e.g. same "places" or "parts of objects" in different images.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def display(img, cmap='gray'):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray')

In [ ]:
reeses = cv2.imread('../data/reeses_puffs.png', 0)
display(reeses)

In [ ]:
cereals = cv2.imread('../data/many_cereals.jpg', 0)
display(cereals)

## Brute-Force Matching with ORB Descriptors

Brute-Force matcher takes the descriptor of one feature in first set and is matched with all other features in second set using some distance calculation. And the closest one is returned.

[ORB (Oriented FAST and Rotated BRIEF) (2011)](https://docs.opencv.org/3.4/d1/d89/tutorial_py_orb.html)

[ORB: an efficient alternative to SIFT or SURF](https://www.gwylab.com/download/ORB_2012.pdf)
 
 
* ORB (Oriented FAST and Rotated BRIEF) is a keypoint detector and descriptor extractor
* computationally-efficient replacement to SIFT that has similar matching performance, is less affected by image noise, and is capable of being used for real-time performance.
* a fusion of FAST keypoint detector and BRIEF descriptor with many modifications to enhance the performance

[cv2::Feature2D Class Reference](https://docs.opencv.org/3.4/d0/d13/classcv_1_1Feature2D.html#a8be0d1c20b08eb867184b8d74c15a677)
* Abstract base class for 2D image feature detectors and descriptor extractors.

[cv2::ORB Class Reference](https://docs.opencv.org/3.4/db/d95/classcv_1_1ORB.html)
* Class implementing the ORB (oriented BRIEF) keypoint detector and descriptor extractor.
* `cv2::ORB` implements `cv2::Feature2D`

[retval = cv2.ORB_create(...)](https://docs.opencv.org/3.4/db/d95/classcv_1_1ORB.html#adc371099dc902a9674bd98936e79739c)
* The ORB constructor.

In [ ]:
orb = cv2.ORB_create()

[keypoints, descriptors = cv.Feature2D.detectAndCompute(image, mask\[, descriptors\[, useProvidedKeypoints\]\])](https://docs.opencv.org/3.4/d0/d13/classcv_1_1Feature2D.html#a8be0d1c20b08eb867184b8d74c15a677)
* Detects keypoints and computes the descriptors

In [ ]:
keyPoints1, descriptors1 = orb.detectAndCompute(reeses, None)
keyPoints2, descriptors2 = orb.detectAndCompute(cereals, None)

In [ ]:
print(f'type(keyPoints1) = {type(keyPoints1)}')
print(f'len(keyPoints1) = {len(keyPoints1)}')
print(f'type(keyPoints1[0]) = {type(keyPoints1[0])}')
print(f'keyPoints1[0] = {keyPoints1[0]}')

print(f'type(descriptors1) = {type(descriptors1)}')
print(f'len(descriptors1) = {len(descriptors1)}')
print(f'type(descriptors1[0]) = {type(descriptors1[0])}')
print(f'descriptors1[0] = {descriptors1[0]}')

print(f'type(keyPoints2) = {type(keyPoints2)}')
print(f'len(keyPoints2) = {len(keyPoints2)}')
print(f'type(keyPoints2[0]) = {type(keyPoints2[0])}')

print(f'type(descriptors2) = {type(descriptors2)}')
print(f'len(descriptors2) = {len(descriptors2)}')
print(f'type(descriptors2[0]) = {type(descriptors2[0])}')

[cv2::BFMatcher Class Reference](https://docs.opencv.org/3.4/d3/da1/classcv_1_1BFMatcher.html)
* Brute-force descriptor matcher.


[retval = cv.BFMatcher_create(\[, normType\[, crossCheck\]\])](https://docs.opencv.org/3.4/d3/da1/classcv_1_1BFMatcher.html#ac6418c6f87e0e12a88979ea57980c020)
* `normType` - One of NORM_L1, NORM_L2, NORM_HAMMING, NORM_HAMMING2. L1 and L2 norms are preferable choices for SIFT and SURF descriptors, NORM_HAMMING should be used with ORB, BRISK and BRIEF, NORM_HAMMING2 should be used with ORB when WTA_K==3 or 4 (see ORB::ORB constructor description).
* `crossCheck` - If it is false, this is will be default BFMatcher behaviour when it finds the k nearest neighbors for each query descriptor. If crossCheck==true, then the knnMatch() method with k=1 will only return pairs (i,j) such that for i-th query descriptor the j-th descriptor in the matcher's collection is the nearest and vice versa, i.e. the BFMatcher will only return consistent pairs. Such technique usually produces best results with minimal number of outliers when there are enough matches. This is alternative to the ratio test, used by D. Lowe in SIFT paper.


In [ ]:
# According to the documentation (https://docs.opencv.org/3.4/d3/da1/classcv_1_1BFMatcher.html#abe0bb11749b30d97f60d6ade665617bd)
# this constructor is obsolete:
# bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

bf = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck=True)  

[matches = cv2.DescriptorMatcher.match(queryDescriptors, trainDescriptors\[, mask\])](https://docs.opencv.org/3.4/db/d39/classcv_1_1DescriptorMatcher.html#a0f046f47b68ec7074391e1e85c750cba)
* Finds the best match for each descriptor from a query set.
* Parameters:
   * `queryDescriptors` - Query set of descriptors.
   * `trainDescriptors` - Train set of descriptors. This set is not added to the train descriptors collection stored in the class object.
   * `matches` - Matches. If a query descriptor is masked out in mask , no match is added for this descriptor. So, matches size may be smaller than the query descriptors count.
   * `mask` - Mask specifying permissible matches between an input query and train matrices of descriptors.

In [ ]:
matches = bf.match(descriptors1, descriptors2)

In [ ]:
type(matches)

In [ ]:
type(matches[0])

[cv::DMatch Class Reference](https://docs.opencv.org/3.4/d4/de0/classcv_1_1DMatch.html)

Class for matching keypoint descriptors. Its public attributes are:
   * queryIdx - query descriptor index
   * trainIdx - train descriptor index
   * imgIdx - train image index
   * distance - distance between descriptors
   
Keypoints are not stored in DMatch, but in the other list. DMatch object only stores the indices of matched keypoints, their distance and the index of the image. You can get this indices to get the keypoints from the other list.

In [ ]:
single_match = matches[0]
single_match.distance

[sorted()](https://docs.python.org/3/library/functions.html#sorted) is Python built-in function that builds a new sorted list from an iterable.

In [ ]:
matches = sorted(matches, key=lambda x:x.distance)

[cv2.drawMatches()](https://docs.opencv.org/master/d4/d5d/group__features2d__draw.html#gad8f463ccaf0dc6f61083abd8717c261a) helps us to draw the matches. It stacks two images horizontally and draw lines from first image to second image showing best matches

```
outImg = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches1to2, outImg[, matchColor[, singlePointColor[, matchesMask[, flags]]]])
```

Parameters:
   * `img1` - First source image.
   * `keypoints1` - Keypoints from the first source image.
   * `img2` - Second source image.
   * `keypoints2` - Keypoints from the second source image.
   * `matches1to2` - Matches from the first image to the second one, which means that keypoints1[i] has a corresponding point in keypoints2[matches[i]] .
   * `outImg` - Output image. Its content depends on the flags value defining what is drawn in the output image. See possible flags bit values below.
   * `matchColor` - Color of matches (lines and connected keypoints). If matchColor==Scalar::all(-1) , the color is generated randomly.
   * `singlePointColor` - Color of single keypoints (circles), which means that keypoints do not have the matches. If singlePointColor==Scalar::all(-1) , the color is generated randomly.
matchesMask	Mask determining which matches are drawn. If the mask is empty, all matches are drawn.
   * `flags` - Flags setting drawing features. Possible flags bit values are defined by DrawMatchesFlags.
This function draws matches of keypoints from two images in the output image. Match is a line connecting two keypoints (circles).


In [ ]:
reeses_matches = cv2.drawMatches(reeses, keyPoints1, cereals, keyPoints2, matches[:25], None, flags=2)

In [ ]:
display(reeses_matches)